In [72]:
from pymongo import MongoClient
import pandas as pd
import pickle
import re
from unidecode import unidecode
from string import whitespace, punctuation

from textblob import TextBlob

In [4]:
client = MongoClient()
db = client.lon_db
db.collection_names()
collection = db.lon_docs

In [11]:
# get the first ten lines of each textbody so I can find the date
doc_list = []

for doc in collection.find({'languageCode':'en'},
                          {'_id':1,
                           'publicationDate':1,'symbol':1,'body_en':1}):
        text = doc['body_en']
        lines_10 = text.split('\n')[:10]
        doc['body_en'] = lines_10
        doc_list.append(doc)

In [17]:
test_list_10 = doc_list[1]['body_en']
print test_list_10

[u'', u'C , 381 .  IL 1 2 5 . 1 9 2 5', u'', u'LEAGT\xeeE_ 0 ?  M A T X O lS', u'', u'G e n e v a , June  2 7 t h ,  1985', u'', u'0 _T h v  : TTJ ,1', u'', u'O p e r a t i o n s  o f  a  S y n d i c a t s  f o r  im po r t i n g  o p i u ra and n a r c o t i c s']


In [27]:
# drop blank lines

no_blanks = []

numbers = re.compile('[0-9]')

for line in test_list_10:
    if re.search(numbers, line) == None:
        pass
    else:
        no_blanks.append(line)

print len(test_list_10)
print len(no_blanks)
no_blanks

10
4


[u'C , 381 .  IL 1 2 5 . 1 9 2 5',
 u'LEAGT\xeeE_ 0 ?  M A T X O lS',
 u'G e n e v a , June  2 7 t h ,  1985',
 u'0 _T h v  : TTJ ,1']

In [31]:
datelike_number =[]

datelike = re.compile('\d{4}')

for line in test_list_10:
    no_space = line.replace(whitespace,'')
    if re.search(datelike, no_space) != None:
        datelike_number.append(line)

In [32]:
print len(no_blanks)
print len(datelike_number)
datelike_number

4
1


[u'G e n e v a , June  2 7 t h ,  1985']

4 numbers in a row works because it is sees the full stop in the document title as a decimal point. That is a neat trick.

In [46]:
def find_dateline(line_list):
    '''
    
    '''
    has_number = []
    for line in line_list:
        if re.search(numbers, line) == None:
            pass
        else:
            has_number.append(line)
        
        
    datelike_number =[]

    datelike = re.compile('\d{4}')

    for line in has_number:
        no_space = line.replace(whitespace,'')
        if re.search(datelike, no_space) != None:
            datelike_number.append(line)
    
    if len(datelike_number) == 0:
        return has_number
    else:
        return datelike_number

In [47]:
# get the first ten lines of each textbody so I can find the date
doc_list = []

for doc in collection.find({'languageCode':'en'},
                          {'_id':1,
                           'publicationDate':1,'symbol':1,'body_en':1}):
        text = doc['body_en']
        lines_10 = text.split('\n')[:10]
        doc['dateline'] = find_dateline(lines_10)
        doc_list.append(doc)

In [48]:
one_dateline = 0
multi_dateline = 0
no_datelike = 0

for doc in doc_list:
    if len(doc['dateline']) == 1:
        one_dateline += 1
    elif len(doc['dateline']) > 1:
        multi_dateline += 1
    else:
        no_datelike += 1

print one_dateline
print multi_dateline
print no_datelike

3973
7654
291


In [50]:
date_list = []

for doc in doc_list:
    date_list.append([doc['symbol'], doc['dateline']])

In [51]:
date_list

[[u'C.476.1930.X', [u'C.476.1930.X.', u'Geneva, September 6th, 1930,']],
 [u'C.381.M.125.1925', [u'G e n e v a , June  2 7 t h ,  1985']],
 [u'C.831.M.412.1931.VII',
  [u'Communicated t o  t h e  ~ C \xbb 8 3 1 . M . 4 1 S . 1 9 3 1 . V I I .',
   u'o f  t h e  L e a g u e  o f  N a t i o n s .  G e n e v a ,  N o v e m b er  1 1 t h ,  1 9 3 1 .',
   u'APPFAL 3 7  THF CHINESE GOVFRNH^NT UNDER APTICLF 11']],
 [u'C.234.1930.II', [u'C.234.1930.II.', u'April 30th, 1930,']],
 [u'C.107.(1).1925.I',
  [u'the C o u n c i l . C .1 07 . (1 )  .1925 \xab1.',
   u'Geneva, March 1 1 th ,  1985.']],
 [u'C.555.1936.XI',
  [u'Communicated to C . 555.1916.XI.',
   u'the Council. O.C.1668.',
   u'Geneva, Lee ember 22nd, 1936.']],
 [u'C.108.1933.VII', []],
 [u'C.661.(1).M.220.1925.VII',
  [u"Comic 11 -3nd r ' .em bo is  ",
   u'6 C  1 .  ( 1 )',
   u'aR\xcfICL.,3 10 and 11 Ox THE COVEITAI\xcfT']],
 [u'C.25.M.25.1945.XI', [u'C.25.M.25.1945.XI ', u'(0.C/A.R.1943/35)']],
 [u'C.429.(1).1934.VII',
  [u'0.4 2

In [65]:
def find_G(line_list):
    '''
    This model will look for a line beginning G
    '''
    
    potential_datelines = []
    
    start_G = re.compile('G')
    for line in line_list:
        line = unidecode(line)
        no_space = line.translate(None, whitespace).translate(None, punctuation)
        if start_G.match(no_space) == None:
            pass
        else:
            potential_datelines.append(line)
    
    return potential_datelines

In [66]:
print find_G(test_list_10)

['G e n e v a , June  2 7 t h ,  1985']


In [68]:
# get the first ten lines of each textbody so I can find the date
doc_list = []

for doc in collection.find({'languageCode':'en'},
                          {'symbol':1,'body_en':1}):
        text = doc['body_en']
        lines_10 = text.split('\n')[:10]
        doc['dateline'] = find_G(lines_10)
        doc_list.append(doc)

In [69]:
one_dateline = 0
multi_dateline = 0
no_datelike = 0

for doc in doc_list:
    if len(doc['dateline']) == 1:
        one_dateline += 1
    elif len(doc['dateline']) > 1:
        multi_dateline += 1
    else:
        no_datelike += 1

print one_dateline
print multi_dateline
print no_datelike

7024
604
4290


In [70]:
date_list = []

for doc in doc_list:
    date_list.append([doc['symbol'], doc['dateline']])

In [71]:
date_list

[[u'C.476.1930.X', ['Geneva, September 6th, 1930,']],
 [u'C.381.M.125.1925', ['G e n e v a , June  2 7 t h ,  1985']],
 [u'C.831.M.412.1931.VII', []],
 [u'C.234.1930.II', ['Geneva,']],
 [u'C.107.(1).1925.I', ['Geneva, March 1 1 th ,  1985.']],
 [u'C.555.1936.XI', ['Geneva, Lee ember 22nd, 1936.']],
 [u'C.108.1933.VII', []],
 [u'C.661.(1).M.220.1925.VII', []],
 [u'C.25.M.25.1945.XI', []],
 [u'C.429.(1).1934.VII', ["Geneva, September 2'/th, 1934*"]],
 [u'C.458.M.174.1924.I', []],
 [u'C.78.M.75.1941.XI', ['Geneva, December 3rd , 1941.']],
 [u'C.116.1926.I', []],
 [u'C.219.M.159.1921.I', []],
 [u'C.74.1930.I', ['G e n e v a ,  J a n u a r y  15 t b , 1930,.']],
 [u'C.831.1924.I', []],
 [u'C.411.1923.I', ['Guseva, June 25th, 1923 v ']],
 [u'C.326.1935.II.A', ['Geneva. September 2nd,1935.']],
 [u'C.772.1925', ['Geneva, December 7th 1925.']],
 [u'C.330.1926.VI', ['Geneva, June 5ths 1926,,']],
 [u'C.667.M.275.1931.IX', ['Geneva, October 5th, 1931.']],
 [u'C.450.1927.VII', []],
 [u'C.374.1931.V

In [ ]:
def find_G_4no(line_list, symbol):
    '''
    This model will look for a line beginning G
    '''
    
    potential_datelines = []
    
    start_G = re.compile('G')
    
    number_3_4 = re.compile('\d{4}')
    
    for line in line_list:
        line = unidecode(line)
        no_space = line.translate(None, whitespace).translate(None, punctuation)
        if start_G.match(no_space) == None:
            pass
        else:
            potential_datelines.append(line.translate(None, whitespace))
    
    if len(potential_datelines) == 1 and re.search('\d{2,3}') != None:
        return potential_datlines
    
    for line in line_list:
        line = unidecode(line)
        

In [ ]:
def find_not_symbol(line_list, symbol):
    
    number_4 = re.compile('\d{2,3}')
    
    has_four_numbers = []
    
    for line in line_list:
        line = unidecode(line)
        nospace = line.translate(None, whitespace)
        if re.search(number_4, line) != None:
            has_four_numbers.append(line)
    
    for line in has_four_numbers:
        
        
    return 
        

In [ ]:
test_symbol = 'C.494.M.335.1937.VII' 

test_list = '[G.S.P.24(1).', 'Geneva, October 21st, 1937.']